In [1]:
# Use IBEIS to extract the trailing edges from humpbacks as our training / validation set, 
# and we'll extract them from humpbacks_fb as our testing set

# DISABLE_DOCTEST
from ibeis_flukematch.plugin import *  # NOQA


+------

<!!! WARNING !!!>
[!controller_inject] Missing flask.ext.cas.
To install try pip install git+https://github.com/cameronbwhite/Flask-CAS.git
<type 'exceptions.ImportError'>: No module named flask.ext.cas
</!!! WARNING !!!>

L______


Trying /home/zach/envs/thesis/local/lib/python2.7/site-packages/flann-1.8.4-py2.7.egg/pyflann/lib/libflann.so
Couldn't import dot_parser, loading of dot files will not be possible.

+------

<!!! WARNING !!!>
[!custom_layers] WARNING: GPU seems unavailable
<type 'exceptions.ImportError'>: No module named pylearn2.sandbox.cuda_convnet.filter_acts
</!!! WARNING !!!>

L______



Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled)
/home/zach/envs/thesis/local/lib/python2.7/site-packages/Theano-0.8.0.dev0-py2.7.egg/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [3]:
ibs = ibeis.opendb(db='humpbacks')
all_aids = ibs.get_valid_aids()
#isvalid = ibs.depc.get_property('Has_Notch', all_aids, 'flag')
#aid_list = ut.compress(all_aids, isvalid)
#aid_list = all_aids
#print('aid_list = %r' % (aid_list,))
depc = ibs.depc
config = {'n_neighbors': 5, 'crop_enabled': True, 'crop_dim_size':1002, 'ignore_notch':False, 'use_te_scorer':True,
          'te_score_weight':0.5,'te_net':'fbannot_simple', 'manual_extract':False, 'te_score_method':'avg'}
bc_list = ibs.depc.get_property('Block_Curvature', all_aids, ('curvature'), config)


[ibs.__init__] new IBEISController
[ibs._init_dirs] ibs.dbdir = '/home/zach/data/IBEIS/humpbacks'
[ensure_daily_database_backup] Daily backup of database: u'/home/zach/data/IBEIS/humpbacks/_ibsdb/_ibeis_database.sqlite3' -> u'/home/zach/data/IBEIS/humpbacks/_ibsdb/_ibeis_backups/_ibeis_database_backup_2016_03_01_00_00_00.sqlite3'
[util_path] [Copying]: 
[C] | /home/zach/data/IBEIS/humpbacks/_ibsdb/_ibeis_database.sqlite3
[C] ->/home/zach/data/IBEIS/humpbacks/_ibsdb/_ibeis_backups/_ibeis_database_backup_2016_03_01_00_00_00.sqlite3
[depc] Initialize ANNOTATIONS depcache
[util_io] * load_cPkl(u'.../humpbacks/general_config.cPkl')
[ibs.__init__] END new IBEISController



In [4]:
# all bc
curvatures = list(bc_list)
ids = ibs.get_annot_nids(all_aids)


In [5]:
all_curvs = np.stack(curvatures, axis=0)
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

curvs, ids = shuffle(all_curvs, ids)
train_curvs, val_curvs, train_ids, val_ids = train_test_split(curvs, ids)

print(train_curvs.shape)
print(len(train_ids))

(3468, 1000, 4)
3468


In [2]:
fb_ibs = ibeis.opendb(db='humpbacks_fb')
fb_all_aids = fb_ibs.get_valid_aids()
#print('aid_list = %r' % (aid_list,))
fb_depc = fb_ibs.depc
fb_config = {'n_neighbors': 5, 'crop_enabled': True, 'crop_dim_size':1002, 'ignore_notch':False, 'use_te_scorer':True,
          'te_score_weight':0.5,'te_net':'fbannot_simple', 'manual_extract':False, 'te_score_method':'avg'}
fb_bc_list = fb_ibs.depc.get_property('Block_Curvature', fb_all_aids, ('curvature'), fb_config)


[ibs.__init__] new IBEISController
[ibs._init_dirs] ibs.dbdir = '/home/zach/data/IBEIS/humpbacks_fb'
[depc] Initialize ANNOTATIONS depcache
[util_io] * load_cPkl(u'.../humpbacks_fb/general_config.cPkl')
[ibs.__init__] END new IBEISController

[deptbl.add] Add 1 / 2423 new props to u'Cropped_Chips' for config_rowid=8
add Cropped_Chips chunk    0/1...  [cropped chip] WARNING: Probably got a bad keypoint prediction: bbox: (959.0, 0, 0.0, 640)
add Cropped_Chips chunk    1/1...  rate=59.45 Hz, etr=0:00:00, ellapsed=0:00:00, wall=19:52 EST
[deptbl.add] finished add
[deptbl.add] Add 2422 / 2422 new props to u'Trailing_Edge' for config_rowid=86
add Trailing_Edge chunk    0/10...  Preprocess Trailing_Edge
<TrailingEdgeConfig(n_nb5,version=5,te_sTrue,w_tes0.5,te_net=fbannot_simple,te_smavg)>
[utool] Already have file /home/zach/.config/ibeis/tescorer_fbannot_simple.pickle
[util_io] * load_cPkl(u'.../ibeis/tescorer_fbannot_simple.pickle')
[score_te] Batch 0/76, processing 32 images
[score_te] Ba

In [12]:
filt_bc, filt_aids = zip(*filter(lambda x: x[0] is not None, zip(list(fb_bc_list), fb_all_aids)))

test_curvs = np.stack(list(filt_bc),axis=0)
test_ids = fb_ibs.get_annot_nids(filt_aids)

In [14]:
from train_utils import save_dataset, dataset_loc
saveto = 'te_match_1000'
save_dataset(join(dataset_loc,'Flukes/patches/%s' % saveto),
             (train_curvs, train_ids),
             (val_curvs, val_ids),
             (test_curvs, test_ids), grey=True)

[util_io] * save_cPkl(u'.../te_match_1000/train.pkl', data)
[util_io] * save_cPkl(u'.../te_match_1000/val.pkl', data)
[util_io] * save_cPkl(u'.../te_match_1000/test.pkl', data)
[util_io] * save_cPkl(u'.../te_match_1000/meanstd.pkl', data)
Took 0.09 seconds
